<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [73]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [74]:
import folium
import pandas as pd

In [75]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [76]:
## Task 1: Mark all launch sites on a map
launch_sites = [
    {"Launch Site": "CCAFS LC-40", "Lat": 28.562302, "Long": -80.577356},
    {"Launch Site": "CCAFS SLC-40", "Lat": 28.563197, "Long": -80.576820},
    {"Launch Site": "KSC LC-39A", "Lat": 28.573255, "Long": -80.646895},
    {"Launch Site": "VAFB SLC-4E", "Lat": 34.632834, "Long": -120.610745}
]

map_launch_sites = folium.Map(location=[28.562302, -80.577356], zoom_start=4)

for site in launch_sites:
    folium.Marker(
        location=[site["Lat"], site["Long"]],
        popup=site["Launch Site"],
        icon=folium.Icon(color='blue', icon='rocket')
    ).add_to(map_launch_sites)

# Näytetään kartta
map_launch_sites

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [77]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [78]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [79]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [80]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [81]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for idx, row in launch_sites_df.iterrows():
    launch_site = row['Launch Site']
    lat = row['Lat']
    long = row['Long']
    
    # Lisää Circle
    folium.Circle(
        location=[lat, long],
        radius=1000,  # Sädettä säädellään tarpeen mukaan
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        popup=launch_site
    ).add_to(site_map)
    
    # Lisää Marker with DivIcon
    folium.Marker(
        location=[lat, long],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12pt; color : black"><b>{launch_site}</b></div>',
        )
    ).add_to(site_map)

# Näytetään kartta
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [82]:
#Answer 1: No, all launch sites are located at relatively higher latitudes, ranging from approximately 28 to 34 degrees north, which are not close to the Equator.
#Answer 2: Yes, all launch sites are near the coastline, facilitating easy access to maritime routes and providing optimal launch trajectories over the ocean.

# Task 2: Mark the success/failed launches for each site on the map
marker_cluster = MarkerCluster().add_to(site_map)
for idx, row in spacex_df.iterrows():
    launch_site = row['Launch Site']
    lat = row['Lat']
    long = row['Long']
    success = row['class']
    
    if success == 1:
        # Succes launches
        folium.Marker(
            location=[lat, long],
            popup=f"{launch_site} - Success",
            icon=folium.Icon(color='green', icon='check'),
        ).add_to(marker_cluster)
    else:
        # Failed launches
        folium.Marker(
            location=[lat, long],
            popup=f"{launch_site} - Failure",
            icon=folium.Icon(color='red', icon='times'),
        ).add_to(marker_cluster)

site_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [83]:
# Calculate success rate per launch site
success_rates = spacex_df.groupby('Launch Site')['class'].mean().reset_index()
success_rates['success_rate'] = success_rates['class'] * 100

# Merge success rates with launch_sites_df
launch_sites_df = launch_sites_df.merge(success_rates[['Launch Site', 'success_rate']], on='Launch Site')

# Define a color function based on success rate
def get_color(rate):
    if rate >= 80:
        return 'green'
    elif rate >= 50:
        return 'orange'
    else:
        return 'red'

# Add CircleMarker with color based on success rate
for idx, row in launch_sites_df.iterrows():
    launch_site = row['Launch Site']
    lat = row['Lat']
    long = row['Long']
    success_rate = row['success_rate']
    
    folium.CircleMarker(
        location=[lat, long],
        radius=5,
        color=get_color(success_rate),
        fill=True,
        fill_color=get_color(success_rate),
        fill_opacity=0.2,
        popup=f"{launch_site}: {success_rate:.2f}% Success"
    ).add_to(site_map)

# Display the enhanced map
site_map

Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [84]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [85]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df['marker_color'] = spacex_df['class'].map({1: 'green', 0: 'red'})

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [86]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# Iterate through each launch record in spacex_df
for index, record in spacex_df.iterrows():
    # Create a Marker object with location and customized icon
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=folium.Popup(
            f"Launch Site: {record['Launch Site']}<br>Success: {'Yes' if record['class'] == 1 else 'No'}",
            parse_html=True
        ),
        icon=folium.Icon(
            color=record['marker_color'],
            icon='rocket' if record['class'] == 1 else 'times',
            prefix='fa'  # Using Font Awesome icons
        )
    )
    # Add the Marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Display the enhanced map with launch outcome markers
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [32]:
# TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [87]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [88]:
# Define the function to calculate distance using the Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # Approximate radius of Earth in kilometers
    R = 6373.0

    # Convert coordinates from decimal degrees to radians
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    # Differences in coordinates
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance

# Coordinates of the launch site (e.g., CCAFS SLC-40)
launch_site_lat = 28.563197
launch_site_lon = -80.576820


*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [89]:
delta_lon = 0.0092

# Coordinates of the closest coastline point (straight east, approximately 0.9 km)
coastline_lat = launch_site_lat  # Same latitude for straight east
coastline_lon = launch_site_lon + delta_lon  # Approximately 0.9 km east

# Calculate the distance between the launch site and the coastline point
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a Folium Marker for the coastline point with the distance displayed
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html=f'''
            <div style="font-size: 12pt; color: #d35400;">
                <b>{distance_coastline:.2f} KM</b>
            </div>
        '''
    )
)

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [90]:
# Add the distance marker to the map
site_map.add_child(distance_marker)

# Define the coordinates for the PolyLine
coordinates = [
    [launch_site_lat, launch_site_lon],
    [coastline_lat, coastline_lon]
]

# Create a folium.PolyLine object using the coastline coordinates and launch site coordinates
polyline = folium.PolyLine(
    locations=coordinates,
    weight=2,        # Thickness of the line
    color='blue',    # Color of the line
    opacity=0.6      # Transparency of the line
)

# Add the PolyLine to the map
site_map.add_child(polyline)

# Adjust the map to fit the PolyLine
site_map.fit_bounds(polyline.get_bounds())

# Display the updated map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

In [94]:
# Coordinates of the closest railway point (replace with your actual coordinates)
railway_lat = 28.563578 # Replace with the railway's latitude
railway_lon = -80.586923 # Replace with the railway's longitude

# Calculate the distance between the launch site and the railway point
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# Create a Folium Marker for the railway point with the distance displayed
railway_marker = folium.Marker(
    location=[railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html=f'''
            <div style="font-size: 12pt; color: #2ecc71;">
                <b>{distance_railway:.2f} KM</b>
            </div>
        '''
    )
)

# Add the railway marker to the map
site_map.add_child(railway_marker)

# Define the coordinates for the PolyLine between the launch site and the railway point
coordinates_railway = [
    [launch_site_lat, launch_site_lon],
    [railway_lat, railway_lon]
]

# Create and add the PolyLine to the map
polyline_railway = folium.PolyLine(
    locations=coordinates_railway,
    weight=2,        # Thickness of the line
    color='green',   # Color of the line
    opacity=0.6      # Transparency of the line
)
site_map.add_child(polyline_railway)

# Adjust the map to fit the PolyLine
site_map.fit_bounds(polyline_railway.get_bounds())

# Display the updated map
site_map


In [ ]:
#Are launch sites in close proximity to railways?
#Yes, launch sites are located near railways.

#Are launch sites in close proximity to highways?
#Yes, launch sites are situated close to highways. 

#Are launch sites in close proximity to coastline?
#Yes, launch sites are near coastlines. 

#Do launch sites keep a certain distance away from cities?
#Yes, launch sites are strategically positioned at a safe distance from major cities.

#Explanation of Findings:
#The analysis of launch site locations reveals a strategic emphasis on accessibility and safety. Proximity to railways and highways ensures efficient logistics and transportation of essential materials and personnel. Being near coastlines allows for safe rocket trajectories over uninhabited ocean areas, mitigating risks to populated regions. Additionally, maintaining a safe distance from cities underscores the priority placed on minimizing potential hazards and ensuring the well-being of nearby populations.

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
